# 3. Get the Context to See Who Else Got the Email

Notebook to build all of the different stix objects associated with finding who else got the suspicious email

We take v3.0 of the [Incident Core Extension](https://github.com/os-threat/cti-stix-common-objects/blob/main/extension-definition-specifications/incident-core/Incident%20Extension%20Suite.adoc) specification, with particular focus on the Sighting Extensions for Evidence, and combine it with the standard Stix approach. There are nine stages:

1. First setup the global parameters and retrieve the context memory
2. Retrieve the user-account, email address and identity for each of the users who recieved the email
2. Add the SCO objects to the `ObservedData` object
4. Create the `Sighting` object with the `SightingContext` extension, using their identity's as the location
5. Add the Sighting to the original Event
5. Create the next Task and its Sequence objects
6. Connect the Sequence object to the previous sequence object
6. Create the Incident, with the [Incident Core Extension](https://github.com/os-threat/cti-stix-common-objects/blob/main/extension-definition-specifications/incident-core/Incident%20Extension%20Suite.adoc), and add all of the objects to it
7. Save the context memory


Each time a block makes an object, and saves it as a json, this notebook will parse the object into an actual Stix object, so it can be bundled and printy printed (This step also verifies the objects are created correctly, and is cool).



## A. Load Imports

### A.1 StixORM Imports


In [1]:
import sys
!{sys.executable} -m pip install stixorm
from stixorm.module.typedb import TypeDBSink, TypeDBSource
from stixorm.module.authorise import import_type_factory
from stixorm.module.definitions.stix21 import (
    Identity, EmailAddress, UserAccount, Relationship, Bundle, Incident, URL, EmailMessage, ObservedData
)
from stixorm.module.definitions.os_threat import (
    EventCoreExt, Event, SocialMediaContact, ContactNumber, IncidentCoreExt, TaskCoreExt,
    Task, SightingEvidence, Sequence, SequenceExt, AnecdoteExt, Anecdote,
    SightingAnecdote, SightingAlert, SightingContext, SightingExclusion,
    SightingEnrichment, SightingHunt, SightingFramework, SightingExternal
)
from stixorm.module.authorise import import_type_factory
from stixorm.module.typedb_lib.instructions import ResultStatus, Result
from stixorm.module.parsing import parse_objects
import_type = import_type_factory.get_all_imports()
import logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

### A.2 Relative Path Imports
https://stackoverflow.com/questions/76162459/jupyter-notebook-importerror-attempted-relative-import-with-no-known-parent-pac/77528726#77528726

In [2]:
import sys
sys.path.append('../')
import os

### A.3 Relative Import of Blocks

In [3]:
import json
from Block_Families.Objects.SCO.Email_Message.make_email_msg import main as make_email_msg
from Block_Families.Objects.SRO.Relationship.make_sro import main as make_sro
from Utilities.local_make_general import invoke_context_get_block, invoke_context_save_block
from Utilities.local_make_sro import invoke_sro_block, invoke_sighting_block
from Utilities.local_make_sdo import (
    invoke_make_observed_data_block, invoke_make_indicator_block, invoke_make_event_block, invoke_make_sequence_block,
    invoke_make_task_block, invoke_make_incident_block, invoke_make_impact_block
)
from Utilities.local_make_sco import (
    invoke_make_email_addr_block, invoke_make_url_block, invoke_make_e_msg_block, invoke_make_anecdote_block
)
from conv import conv

context_base = "../Orchestration/Context_Mem/"
path_base = "../Block_Families/Objects/"
results_base = "../Orchestration/Results/"
sro_data = "SRO/Relationship/sro_derived.json"
context_data = {
    "usr2": "sbilly@example.com",
    "usr3": "wwhilly@example.com"
}

TR_Context_Memory_Path = "./Context_Mem/Type_Refinery_Context.json"

ImportError: cannot import name 'invoke_context_get_block' from 'Utilities.local_make_general' (C:\Users\brett\PycharmProjects\brett_blocks\Orchestration\Utilities\local_make_general.py)

### A.4 Load the Context Memory

In [ ]:
# 1. Load the Context
Type_Refinery_Context = invoke_context_get_block()
#
# 2. Setup the  TR User Context
#
local = Type_Refinery_Context["local"]
local_context = local["context"]
me = local_context["me"]
team = local_context["team"]
company = local_context["company"]
systems = local_context["systems"]
assets = local_context["assets"]
#
# 3. Setup the Incident Context
#
local_incident = local["incident"]
sequence_start_objs = local_incident["sequence_start_objs"]
sequence_objs = local_incident["sequence_objs"]
task_objs = local_incident["task_objs"]
event_objs = local_incident["event_objs"]
impact_objs = local_incident["impact_objs"]
other_object_objs = local_incident["other_object_objs"]
incident_obj = local_incident["incident_obj"]

## B. Collect the SCO's Based on Searching the Exchange Server

### B.1 Retrieve the User Account, the Email Address, and the Identity from the Context Memory

for each user who got the phishing email

In [ ]:
# 1. For Naive Smith, find the user account and the email. Plus get my own identity
usr2_email = context_data["usr2"]
usr3_email = context_data["usr3"]
# 2. Setup variables to hold the dicts
usr2_email_addr = {}
usr2_user_account = {}
usr2_identity = {}
usr3_email_addr = {}
usr3_user_account = {}
usr3_identity = {}
#
# 3. Retrieve the email addresses for the employees involved
#
for rec in company:
    if rec["type"] == "email-addr":
        if rec["value"] == usr2_email:
            usr2_email_addr = rec
        if rec["value"] == usr3_email:
            usr3_email_addr = rec
#
# 4. Retrieve the user accounts for the employees involved
#
for rec in company:
    if rec["type"] == "user-account":
        if rec["id"] == usr2_email["belongs_to_ref"]:
            usr2_email_addr = rec
        if rec["id"] == usr3_email["belongs_to_ref"]:
            usr3_email_addr = rec
#
# 5. Retrieve the identity's for the employees involved
#
for rec in company:
    if rec["type"] == "identity" and rec["identity_class"] == "individual":
        for email_address in rec["extensions"]["extension-definition--66e2492a-bbd3-4be6-88f5-cc91a017a498"]["email_addresses"]:
            if email_address["email_address_ref"] == usr2_email["id"]:
                usr2_identity = rec
            if email_address["email_address_ref"] == usr3_email["id"]:
                usr3_identity = rec

## B Make the Anecdote

In [ ]:
# 1. Setup the form paths and the results path
anecdote_form = "SCO/Anecdote/anecdote_on_impact.json"
results_path = "step2/anecdote.json"
# 2. Invoke the Make Anecdote block
anecdote_list = invoke_make_anecdote_block(anecdote_form, results_path, anecdote_reporter=reporter_identity)
# 3. Add the record to the in-session bundles and lists
bundle_list = bundle_list + anecdote_list
other_object_objs.append(anecdote_list[0])
# recover the Anecdote object
anecdote = conv(anecdote_list[0])

## C. Collect the Elements of the Observation into an Observed-Data Object

Tbis then represents the potential phishing email as a group of elements

In [ ]:
# 1. Setup0 list of obsrvations
observed_ids = []
observed_ids.append(anecdote["id"])
# 2. Setup path to form and results
obs_path ="SDO/Observed_Data/observation-alert.json"
results_path ="step1/observation-anecdote.json"
# 3. Invoke the Make Observed Data Block
obs_2 = invoke_make_observed_data_block(obs_path, results_path, observation=observed_ids)
# 4. Add the record to the in-session bundles and lists
bundle_list = bundle_list + obs_2
other_object_objs.append(obs_2[0])
# recover the observation object
observe_2 = conv(obs_2[0])

## D. Create the Anecdote Sighting

Tbis then represents the Anecdote on Impact from the reporting user

### D.1 Create the Sighting
Connecting the:
- observed-data object
- the reporter identity as the sighted object
- the laptop identity as the location

In [ ]:
# 1. Setup path to form and results
sighting_data_path ="SRO/Sighting/sighting_anecdotet.json"
results_path ="step2/sighting_anecdote.json"
# 2. Setup the SDO sighted, the Observed-Data that was observed with generated objects,
#                        then the identity object from the context storage (note the slight difference in indexing #
observed_list = [observe_2["id"]]
sighted = reporter_identity["id"]
TR_where_list = [TR_identity["id"]]
# 2. Invoke the Make Observed Data Block
sight2 = invoke_sighting_block(sighting_data_path, results_path, observed=observed_list, sighted=sighted, where=TR_where_list)
# 3. Add the record to the in-session bundles and lists
bundle_list = bundle_list + sight2
other_object_objs.append(sight2[0])

## E. Create the Impact Objects

Tbis Impact represents the effect reported by the user in the Anecdote

In [ ]:
# 1. Setup path to form and results
impact_path ="SDO/Impact/anecdote_impact.json"
results_path ="step2/impact_anecdote.json"
# 2. Setup the number of assets impacted
numbers = {"computers-mobile": 1}
impacted = reporter_laptop["id"]
# 2. Invoke the Make Observed Data Block
impact_1 = invoke_make_impact_block(impact_path, results_path, impacted_entity_counts=numbers, impacted_refs=impacted, superseded_by_ref=None)
# 3. Add the record to the in-session bundles and lists
bundle_list = bundle_list + impact_1
impact_objs.append(impact_1[0])
other_object_objs.append(reporter_laptop)

## F. Create the Next Task Object

Next step is to investigate the Exchange server to see who else got the email

### F.1 Create the Task Object

In [ ]:
# 1. Setup path to form and results
task_data_path ="SDO/Task/task_anecdote.json"
results_path ="step2/task_anecdote.json"
# 2. Invoke the Make Observed Data Block
task_2 = invoke_make_task_block(task_data_path, results_path, changed_objects=None)
# 3. Add the record to the in-session bundles and lists
bundle_list = bundle_list + task_2
task_objs.append(task_2[0])

### F.2 Create the Sequence Object

In [ ]:
# 1. Setup path to form and results
sequence_data_path ="SDO/Sequence/sequence_alert.json"
results_path ="step1/sequence_task_single.json"
# 2. Setup the Sequence Object for the Event
#
step_type = "single_step"
sequence_type = "task"
sequenced_object = task_2[0]["id"]
# 2. Invoke the Make Observed Data Block
seq_T_2 = invoke_make_sequence_block(sequence_data_path, results_path, step_type=step_type, sequence_type=sequence_type, sequenced_object=sequenced_object, on_completion=None, on_success=None, on_failure=None, next_steps=None)
# 3. Add the record to the in-session bundles and lists
bundle_list = bundle_list + seq_T_2
sequence_objs.append(seq_T_2[0])

### F.3 Connect the Sequence to the Previous one

In [ ]:
# Find the Previous Sequence and link it to the above Sequence
last_task = task_objs[-1]
for rec in sequence_objs:
    if rec["sequence_type"] == "task" and rec["sequenced_object"] == last_task["id"]:
        rec["on_completion"] = seq_T_2[0]["id"]

## G. Finally, Append all of the objects to the Incident Object

Assign all of the objets to the Incident Object

In [ ]:
# 1. Setup path to form and results
inc_path ="SDO/Incident/phishing_incident.json"
results_path ="step1/incident_alert.json"
# 2. Setup the Sequence Object for the Event
#
sequence_start_refs = [x["id"] for x in sequence_start_objs]
sequence_refs = [x["id"] for x in sequence_objs]
task_refs = [x["id"] for x in task_objs]
event_refs = [x["id"] for x in event_objs]
impact_refs = [x["id"] for x in impact_objs]
other_object_refs = [x["id"] for x in other_object_objs]
# 3. Update the actual Incident Object
incident_obj["sequence_start_refs"] = sequence_start_refs
incident_obj["sequence_refs"] = sequence_refs
incident_obj["task_refs"] = task_refs
incident_obj["event_refs"] = event_refs
incident_obj["impact_refs"] = impact_refs
incident_obj["other_object_refs"] = other_object_refs

## H. Write out the Contex Memory for the Incident

Export out the Context Memory for the Incident

In [ ]:
# Save the Tpe Refinery Context Memory File
invoke_context_save_block(Type_Refinery_Context)